In [31]:
import numpy as np
import statsmodels.api as sm
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.express as px
yf.pdr_override()


In [32]:
def getStockData(stocks,start,end):
    stock_symbols = ','.join(stocks)
    stockData=pdr.get_data_yahoo(stock_symbols,start=start,end=end,interval='1mo')
    stockData=stockData['Adj Close']
    return stockData

In [33]:
def getMonthlyReturns(stock_data):
    monthlyReturns=np.log(stock_data/stock_data.shift(1))
    monthlyReturns=monthlyReturns.dropna()
    return monthlyReturns

Calculating the monthly returns

In [34]:
stocks=['WIPRO.BO','^BSESN']
start='2018-04-01'
end='2023-03-31' 
stock_data=getStockData(stocks,start,end)
getMonthlyReturns(stock_data)

[*********************100%%**********************]  2 of 2 completed


,WIPRO.BO,^BSESN
Date,,
2018-05-01,-0.062352,0.004597
2018-06-01,-0.001911,0.002858
2018-07-01,0.057243,0.059804
2018-08-01,0.085972,0.027240
2018-09-01,0.072093,-0.064611
2018-10-01,0.022267,-0.050530
2018-11-01,-0.020264,0.049623
2018-12-01,0.016486,-0.003487
2019-01-01,0.111759,0.005209


In [35]:
BSE_Returns=getMonthlyReturns(stock_data)['^BSESN']
WIPRO_Returns=getMonthlyReturns(stock_data)['WIPRO.BO']

Linear Regression

In [36]:
X=np.array([getMonthlyReturns(stock_data)['^BSESN']])
X=X.reshape(-1,1)

y=np.array([getMonthlyReturns(stock_data)['WIPRO.BO']])
y=y.reshape(-1,1)

X=sm.add_constant(X)

model=sm.OLS(y,X).fit()

In [37]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     5.557
Date:                Sat, 20 Apr 2024   Prob (F-statistic):             0.0219
Time:                        02:37:59   Log-Likelihood:                 66.127
No. Observations:                  59   AIC:                            -128.3
Df Residuals:                      57   BIC:                            -124.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0057      0.011      0.534      0.5

In [38]:
Beta_Levered_TopDown=model.params[1].round(3)
print("Levered Beta for Wipro calculated using Top Down Approach is: ",Beta_Levered_TopDown)

Levered Beta for Wipro calculated using Top Down Approach is:  0.439


# CAPM Model

The CAPM (Capital Asset Pricing Model) is represented by the following equation:

$$
E(r_s) = r_f + \beta (E(r_m) - r_f)
$$

Where:
- E($r_s$) is the expected return on the investment (Cost of Equity).
- E($r_m$) is the expected return of the market.
- $r_f$ is the risk-free rate.
- $\beta$ is the beta, which measures the sensitivity of the investment's returns to market returns.

This equation helps in estimating the expected return on an investment based on its risk, market conditions, and the risk-free rate.


In [39]:
Expected_market_return=getMonthlyReturns(stock_data)['^BSESN'].mean()
Risk_Free_Rate=0.067378 # As of Feb 25 2023

Cost_of_Equity = Risk_Free_Rate + Beta_Levered_TopDown*(Expected_market_return - Risk_Free_Rate)
print("Cost of equity for Wipro calculated using Top Down Approach is: ",Cost_of_Equity)

Cost of equity for Wipro calculated using Top Down Approach is:  0.04164941838782414


# BOTTOM-UP Approach

## Comparable companies in IT industry BSE
* Infosys Limited
* HCL Technologies Limited
* LTIMindtree Limited
* Tech Mahindra Limited

In [40]:
stocks=['INFY.BO','HCLTECH.BO','LTIM.BO','TECHM.BO','^BSESN']
stock_data=getStockData(stocks,start,end)
getMonthlyReturns(stock_data)

[*********************100%%**********************]  5 of 5 completed


,HCLTECH.BO,INFY.BO,LTIM.BO,TECHM.BO,^BSESN
Date,,,,,
2018-05-01,-0.143774,0.026736,0.102024,0.058419,0.004597
2018-06-01,0.022095,0.058986,-0.039001,-0.081161,0.002858
2018-07-01,0.040075,0.043611,0.112315,0.035687,0.059804
2018-08-01,0.081238,0.053489,-0.039972,0.139227,0.027240
2018-09-01,0.039030,0.010844,0.062609,-0.026036,-0.064611
2018-10-01,-0.034881,-0.058853,-0.081550,-0.001074,-0.050530
2018-11-01,-0.035254,-0.029202,-0.115884,-0.053336,0.049623
2018-12-01,-0.052467,-0.010028,0.094609,0.021871,-0.003487
2019-01-01,0.043356,0.127527,0.018377,0.016163,0.005209


In [41]:
X=np.array([getMonthlyReturns(stock_data)['^BSESN']])
X=X.reshape(-1,1)

y1=np.array([getMonthlyReturns(stock_data)['INFY.BO']])
y1=y1.reshape(-1,1)

y2=np.array([getMonthlyReturns(stock_data)['HCLTECH.BO']])
y2=y2.reshape(-1,1)

y3=np.array([getMonthlyReturns(stock_data)['LTIM.BO']])
y3=y3.reshape(-1,1)

y4=np.array([getMonthlyReturns(stock_data)['TECHM.BO']])
y4=y4.reshape(-1,1)

X=sm.add_constant(X)

model_infosys=sm.OLS(y1,X).fit()
model_hcl=sm.OLS(y2,X).fit()
model_ltim=sm.OLS(y3,X).fit()
model_techm=sm.OLS(y4,X).fit()

In [42]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='INFY.BO')
debt_to_equity_infosys=(df.iloc[3,11])/(df.iloc[3,3])
print('Debt to Equity of Infosys: ',debt_to_equity_infosys)


Debt to Equity of Infosys:  0.10949696444058976


In [43]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='HCLTECH.BO')
debt_to_equity_hcl=(df.iloc[3,11])/(df.iloc[3,3])
print('Debt to Equity Ratio of HCL Technologies Limited: ',debt_to_equity_hcl)


Debt to Equity Ratio of HCL Technologies Limited:  0.0731826661365791


In [44]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='LTIM.BO')
debt_to_equity_ltim=(df.iloc[3,11])/(df.iloc[3,3])
print('Debt to Equity Ratio of LTIMindTree: ',debt_to_equity_ltim)


Debt to Equity Ratio of LTIMindTree:  0.09284784808906453


In [45]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='TECHM.BO')
debt_to_equity_techm=(df.iloc[3,11])/(df.iloc[3,3])
print('Debt to Equity Ratio of Tech Mahindra: ',debt_to_equity_techm)


Debt to Equity Ratio of Tech Mahindra:  0.09650392502826231


In [46]:
debt_to_equity_comparable=[debt_to_equity_infosys,debt_to_equity_hcl,debt_to_equity_ltim,debt_to_equity_techm]
Beta_Levered_comparable=[model_infosys.params[1],model_hcl.params[1],model_ltim.params[1],model_techm.params[1]]
Beta_Unlevered_comparable=[]
print("Levered Beta of the Comparable companies are :",Beta_Levered_comparable)
TaxRate=0.25
for i in range(len(Beta_Levered_comparable)):
    unlevered=(Beta_Levered_comparable[i])/(1+debt_to_equity_comparable[i]*(1-TaxRate))
    Beta_Unlevered_comparable.append(unlevered)
print("Unlevered Beta of the Comparable companies are :",Beta_Unlevered_comparable)
Industry_Beta=np.sum(Beta_Unlevered_comparable)/4
print("The Beta of the industry is :",Industry_Beta)

Levered Beta of the Comparable companies are : [0.6519227859353003, 0.8191493419359441, 0.9851852764631174, 0.8992671338957143]
Unlevered Beta of the Comparable companies are : [0.6024481067441978, 0.776528047311859, 0.9210473295597646, 0.8385729482026559]
The Beta of the industry is : 0.7846491079546194


In [47]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='WIPRO.BO')
debt_to_equity_wipro=(df.iloc[3,11])/(df.iloc[3,3])
print('Debt to Equity Ratio of Wipro: ',debt_to_equity_wipro)


Debt to Equity Ratio of Wipro:  0.22342862771233368


In [48]:
# Relevering the Beta of Wipro
Beta_Levered_Wipro=(Industry_Beta)*(1+(debt_to_equity_wipro)*(1-TaxRate))
print(Beta_Levered_Wipro)

0.9161339130241251


In [49]:
ticker_symbol = "WIPRO.BO"
company = yf.Ticker(ticker_symbol)
income_statement = company.financials
ebit = income_statement.loc["EBIT"]
interest_expense = income_statement.loc["Interest Expense"]
interest_coverage_ratio = ebit/interest_expense
print(f"Interest Expense Ratio in FY 2023 for {ticker_symbol} is: {interest_coverage_ratio.iloc[0]}")


Interest Expense Ratio in FY 2023 for WIPRO.BO is: 15.652872878832985


#### Large Cap
The market cap of these companies is significantly high, coming in at around Rs. 20,000 crores or more.
#### Mid Cap
The market cap generally tends to range from Rs. 5,000 to Rs. 20,000 crores. 
#### Small Cap
The market cap is generally below Rs. 5,000 crores


| Interest Coverage Ratio | Rating | Spread   |
|-------------------------|--------|----------|
| > 8.5                   | AAA    | 0.75%    |
| 6.5 - 8.5              | AA     | 1.00%    |
| 5.5 - 6.5              | A+     | 1.50%    |
| 4.25 - 5.5             | A      | 1.80%    |
| 3 - 4.25               | A-     | 2.00%    |
| 2.5 - 3                | BBB    | 2.25%    |
| 2 - 2.5                | BB     | 3.50%    |
| 1.75 - 2               | B+     | 4.75%    |
| 1.5 - 1.75             | B      | 6.50%    |
| 1.25 - 1.5            | B-     | 8.00%    |
| 0.8 - 1.25             | CCC    | 10.00%   |
| 0.65 - 0.8            | CC     | 11.50%   |
| 0.2 - 0.65             | C      | 12.70%   |
| < 0.2                  | D      | 14.00%   |


The market Cap of Wipro is 2.043T=204300 crore( as of 23/10/2023 ) which is greater than 20000 thus Wipro is a Large Cap company

In [50]:
print(f"Interest Coverage Ratio: {interest_coverage_ratio[0]}")
spread=0.0075
print(f"Spread: {spread}")

Interest Coverage Ratio: 15.652872878832985
Spread: 0.0075


C:\Users\Rama Balasubramanian\AppData\Local\Temp\ipykernel_21588\349957413.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Interest Coverage Ratio: {interest_coverage_ratio[0]}")


In [51]:
Cost_of_Debt=Risk_Free_Rate+spread
print(Cost_of_Debt)

0.074878


In [55]:
df = pd.read_excel('bal_sheets.xlsx', sheet_name='WIPRO.BO')
mvd_wipro=df.iloc[3,11]
mve_wipro=(1/debt_to_equity_wipro)*mvd_wipro
print("Market Value of Equity for Wipro is: ",mve_wipro)

Market Value of Equity for Wipro is:  781753000.0000001


## WACC Formula

$$ R_{\text{WACC}} = R_s \cdot \left(\frac{E}{V}\right) + R_d \cdot \left(1 - T\right) \cdot \left(\frac{D}{V}\right) $$

Where:
- $R_{\text{WACC}}$ is the weighted average cost of capital.
- $R_s$ is the cost of equity.
- $ R_d$ is the cost of debt.
- $ E$ is the market value of equity.
- $ D$ is the market value of debt.
- $ V$ is the total value of the firm.
- $ T$ is the Corporate tax rate.


In [53]:
debt_weight=1/(1+(1/debt_to_equity_wipro))
equity_weight=1/(1+debt_to_equity_wipro)
Cost_of_Equity_Wipro=Risk_Free_Rate+Beta_Levered_Wipro*(Expected_market_return-Risk_Free_Rate)

R_wacc=Cost_of_Equity_Wipro*(equity_weight)+Cost_of_Debt*(1-TaxRate)*debt_weight
print("Cost of Capital to Wipro: ",R_wacc)

Cost of Capital to Wipro:  0.021442468400334064
